In [ ]:
!git clone https://github.com/castorini/rank_llm.git

In [ ]:
!pip install -q -e "./rank_llm[genai]" langchain langchain-google-genai langchain-community langchain-core cohere gitpython rank_bm25 faiss-cpu

In [ ]:
!pip list | grep -E 'rank-llm|langchain|faiss|dotenv'

In [ ]:

import os
import time
import warnings
from google.colab import userdata

warnings.filterwarnings("ignore", message="Failed to load GPU Faiss")

from langchain.text_splitter import RecursiveCharacterTextSplitter, Language
from langchain_community.document_loaders import GitLoader
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_google_genai import GoogleGenerativeAIEmbeddings

import google.generativeai as genai
from rank_llm.rerank.rankllm import PromptMode
from rank_llm.rerank.listwise.rank_gemini import SafeGenai
from rank_llm.data import Query, Candidate, Request


google_api_key = userdata.get('GOOGLE_API_KEY')
print("Initializing all AI components...")

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004",
    google_api_key=google_api_key
)

genai.configure(api_key=google_api_key)


native_gemini_model = genai.GenerativeModel("gemini-2.5-flash")

reranker = SafeGenai(
    model="gemini-2.5-flash",
    context_size=8192,
    keys=[google_api_key],
    prompt_mode=PromptMode.RANK_GPT_APEER,
)


import shutil

repo_path = "/tmp/requests_repo"
if os.path.exists(repo_path):
    shutil.rmtree(repo_path)


loader = GitLoader(
    clone_url="https://github.com/Nikhils-G/Agentic-RAG-Chatbot",
    repo_path="/tmp/requests_repo",
    branch="main",
    file_filter=lambda file_path: file_path.endswith(".py")
)
row_docs = loader.load()
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=1000, chunk_overlap=200
)
chunks = python_splitter.split_documents(row_docs)
vector_store = FAISS.from_documents(chunks, embeddings)
vector_retriever = vector_store.as_retriever(search_kwargs={"k": 20})
bm25_retriever = BM25Retriever.from_documents(chunks)
bm25_retriever.k = 20
ensemble_retriever = EnsembleRetriever(
    retrievers=[vector_retriever, bm25_retriever], weights=[0.5, 0.5], search_type="mmr"
)

prompt_template_str = """You are an expert programming assistant.
Your job is to answer questions about a Python codebase.
Use the following retrieved and reranked code snippets to answer the user's question.
If you don't know the answer from the context provided, just say that you don't know.
Do not make up an answer. Provide code examples from the context where relevant.

CONTEXT:
{context}

QUESTION:
{question}

ANSWER:
"""

def ask_with_reranker(question: str):
    start_time = time.time()


    retrieved_lc_docs = ensemble_retriever.invoke(question)

    if not retrieved_lc_docs:
        print("\n✅ No documents retrieved.")
        return


    query = Query(text=question, qid=1)
    candidates_to_rerank = retrieved_lc_docs[:100]
    rank_llm_candidates = [
        Candidate(docid=f"doc_{i}", score=0.0, doc={"text": lc_doc.page_content, "metadata": lc_doc.metadata})
        for i, lc_doc in enumerate(candidates_to_rerank)
    ]
    single_request = Request(query=query, candidates=rank_llm_candidates)


    batch_results = reranker.rerank_batch(requests=[single_request], top_k=5)
    rerank_results = batch_results[0]

    print("      4. Generating final answer...")
    reranked_docs_content = [result.doc["text"] for result in rerank_results.candidates]
    context_str = "\n\n---\n\n".join(reranked_docs_content)
    final_prompt = prompt_template_str.format(context=context_str, question=question)
    native_response = native_gemini_model.generate_content(final_prompt)
    response = native_response.text

    end_time = time.time()
    print(response)


    print("Original order (first 3):")
    for i in range(min(3, len(candidates_to_rerank))):
        source = candidates_to_rerank[i].metadata.get('source', 'N/A')
        print(f"  {i+1}. doc_{i} - {source}")

    print("Reranked order (first 3):")
    for i in range(min(3, len(rerank_results.candidates))):
        candidate_id = rerank_results.candidates[i].docid
        # Extract original index from docid (e.g., "doc_5" -> 5)
        original_idx = int(candidate_id.split('_')[1])
        source = rerank_results.candidates[i].doc["metadata"].get('source', 'N/A')
        print(f"  {i+1}. {candidate_id} (was #{original_idx+1}) - {source}")


my_question = "Parser?"
ask_with_reranker(my_question)